In [1]:
#@title Copyright 2022 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_asset_from_cloud_geotiff.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_asset_from_cloud_geotiff.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Cloud GeoTiff Backed Earth Engine Assets

***Note:*** *The REST API contains new and advanced features that may not be suitable for all users.  If you are new to Earth Engine, please get started with the [JavaScript guide](https://developers.google.com/earth-engine/guides/getstarted).*

Earth Engine can load images from Cloud Optimized GeoTiffs (COGs) in Google Cloud Storage ([learn more](https://developers.google.com/earth-engine/guides/image_overview#images-from-cloud-geotiffs)).  This notebook demonstrates how to create Earth Engine assets backed by COGs.  An advantage of COG-backed assets is that the spatial and metadata fields of the image will be indexed at asset creation time, making the image more performant in collections.  (In contrast, an image created through `ee.Image.loadGeoTIFF` and put into a collection will require a read of the GeoTiff for filtering operations on the collection.)  A disadvantage of COG-backed assets is that they may be several times slower than standard assets when used in computations.

To create a COG-backed asset, make a `POST` request to the Earth Engine [`CreateAsset` endpoint](https://developers.google.com/earth-engine/reference/rest/v1alpha/projects.assets/create).  As shown in the following, this request must be authorized to create an asset in your user folder.

In [1]:
import pandas as pd
from google.cloud import storage
import os
import json

## Start an authorized session

To be able to make an Earth Engine asset in your user folder, you need to be able to authenticate as yourself when you make the request.  You can use credentials from the Earth Engine authenticator to start an [`AuthorizedSession`](https://google-auth.readthedocs.io/en/master/reference/google.auth.transport.requests.html#google.auth.transport.requests.AuthorizedSession).  You can then use the `AuthorizedSession` to send requests to Earth Engine.

In [2]:
import ee
from google.auth.transport.requests import AuthorizedSession

ee.Authenticate()  #  or !earthengine authenticate --auth_mode=gcloud
session = AuthorizedSession(ee.data.get_persistent_credentials())

Enter verification code:  4/1AVHEtk46Xx160lt69AxqBkjxaCSXp0RaKnfcu8MP4M3Uz-qfoYaAxO6sYDM



Successfully saved authorization token.


## Get image information from gcp
Get list of images and their respective uris from the gcp data bucket


In [3]:
def list_blobs_with_prefix(bucket_name, prefix, delimiter=None):
    

    storage_client = storage.Client(project='swhm-prod')

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)

    # Note: The call returns a response only when the iterator is consumed.
    blob_list = []
    for blob in blobs:
        blob_list.append(blob.name)

    if delimiter:
        print("Prefixes:")
        for prefix in blobs.prefixes:
            blob_list.append([prefix])
    
    return blob_list


In [4]:
BUCKET_NAME = 'live_data_layers'
blobsout = list_blobs_with_prefix(BUCKET_NAME,'raster')

In [5]:
df = pd.DataFrame(blobsout, columns=['blob_name'])
df['uri'] = 'gs://'+BUCKET_NAME+'/'+df['blob_name']
(df)

,blob_name,uri
0,rasters/Age_of_Imperviousness.tif,gs://live_data_layers/rasters/Age_of_Imperviou...
1,rasters/Flow_Duration_Index.tif,gs://live_data_layers/rasters/Flow_Duration_In...
2,rasters/HSPF_Land_Cover_Type.tif,gs://live_data_layers/rasters/HSPF_Land_Cover_...
3,rasters/Hydrologic_Response_Units.tif,gs://live_data_layers/rasters/Hydrologic_Respo...
4,rasters/Imperviousness.tif,gs://live_data_layers/rasters/Imperviousness.tif
5,rasters/Land_Cover.tif,gs://live_data_layers/rasters/Land_Cover.tif
6,rasters/Land_Use.tif,gs://live_data_layers/rasters/Land_Use.tif
7,rasters/Landcover.tif,gs://live_data_layers/rasters/Landcover.tif
8,rasters/Population_Density.tif,gs://live_data_layers/rasters/Population_Densi...
9,rasters/Precipitation_mm.tif,gs://live_data_layers/rasters/Precipitation_mm...


In [6]:

file_names = [os.path.splitext(os.path.basename(file_path))[0] for file_path in df['blob_name']]
df["asset_name"] = file_names

pretty_names  = [os.path.splitext(os.path.basename(file_path))[0].replace('_', ' ') for file_path in df['blob_name']]

df["pretty_name"] = pretty_names
df

,blob_name,uri,asset_name,pretty_name
0,rasters/Age_of_Imperviousness.tif,gs://live_data_layers/rasters/Age_of_Imperviou...,Age_of_Imperviousness,Age of Imperviousness
1,rasters/Flow_Duration_Index.tif,gs://live_data_layers/rasters/Flow_Duration_In...,Flow_Duration_Index,Flow Duration Index
2,rasters/HSPF_Land_Cover_Type.tif,gs://live_data_layers/rasters/HSPF_Land_Cover_...,HSPF_Land_Cover_Type,HSPF Land Cover Type
3,rasters/Hydrologic_Response_Units.tif,gs://live_data_layers/rasters/Hydrologic_Respo...,Hydrologic_Response_Units,Hydrologic Response Units
4,rasters/Imperviousness.tif,gs://live_data_layers/rasters/Imperviousness.tif,Imperviousness,Imperviousness
5,rasters/Land_Cover.tif,gs://live_data_layers/rasters/Land_Cover.tif,Land_Cover,Land Cover
6,rasters/Land_Use.tif,gs://live_data_layers/rasters/Land_Use.tif,Land_Use,Land Use
7,rasters/Landcover.tif,gs://live_data_layers/rasters/Landcover.tif,Landcover,Landcover
8,rasters/Population_Density.tif,gs://live_data_layers/rasters/Population_Densi...,Population_Density,Population Density
9,rasters/Precipitation_mm.tif,gs://live_data_layers/rasters/Precipitation_mm...,Precipitation_mm,Precipitation mm


## Build the manifest

In [7]:
def get_layer_dict(asset): 
    data = json.load(open("data/rasters.json"))
    layer_dict = data[asset]
    print(asset)
    #delete dictionaries 
    try: 
        del layer_dict["layer"]
        del layer_dict["values"]
        del layer_dict["labels"]

    except KeyError:
        pass
    

    layer_dict['pretty_name'] = asset
    return layer_dict

In [8]:
get_layer_dict('Runoff mm')

Runoff mm


{'discrete': 'FALSE',
 'sourceName': 'The Nature Conservancy',
 'units': 'mm/year',
 'scale': 10,
 'default_reduction': 'mean',
 'description': 'Mean annual runoff calculated through continuous simulation for the period 1970-1999.',
 'safe_name': 'runoff_mm',
 'pretty_name': 'Runoff mm'}

Create json_data

In [9]:
df['json_data'] = 'na'
for index, row in df.iterrows():
    key_value = row['pretty_name']
    try: 
        json_value = json.dumps(get_layer_dict(key_value))
    except KeyError:
        json_value = "NA"
    df.at[index, 'json_data'] = json_value


Age of Imperviousness
Flow Duration Index
HSPF Land Cover Type
Hydrologic Response Units
Imperviousness
Land Cover
Land Use
Population Density
Runoff mm
Slope Categories
Total Copper Concentration
Total Kjeldahl Nitrogen Concentration
Total Phosphorus Concentration
Total Suspended Solids Concentration
Traffic


In [10]:
df

,blob_name,uri,asset_name,pretty_name,json_data
0,rasters/Age_of_Imperviousness.tif,gs://live_data_layers/rasters/Age_of_Imperviou...,Age_of_Imperviousness,Age of Imperviousness,"{""discrete"": ""TRUE"", ""sourceName"": ""The Nature..."
1,rasters/Flow_Duration_Index.tif,gs://live_data_layers/rasters/Flow_Duration_In...,Flow_Duration_Index,Flow Duration Index,"{""discrete"": ""FALSE"", ""sourceName"": ""The Natur..."
2,rasters/HSPF_Land_Cover_Type.tif,gs://live_data_layers/rasters/HSPF_Land_Cover_...,HSPF_Land_Cover_Type,HSPF Land Cover Type,"{""discrete"": ""TRUE"", ""sourceName"": ""The Nature..."
3,rasters/Hydrologic_Response_Units.tif,gs://live_data_layers/rasters/Hydrologic_Respo...,Hydrologic_Response_Units,Hydrologic Response Units,"{""discrete"": ""TRUE"", ""sourceName"": ""The Nature..."
4,rasters/Imperviousness.tif,gs://live_data_layers/rasters/Imperviousness.tif,Imperviousness,Imperviousness,"{""discrete"": ""FALSE"", ""sourceName"": ""The Natur..."
5,rasters/Land_Cover.tif,gs://live_data_layers/rasters/Land_Cover.tif,Land_Cover,Land Cover,"{""discrete"": ""TRUE"", ""sourceName"": ""The Nature..."
6,rasters/Land_Use.tif,gs://live_data_layers/rasters/Land_Use.tif,Land_Use,Land Use,"{""discrete"": ""TRUE"", ""sourceName"": ""Puget Soun..."
7,rasters/Landcover.tif,gs://live_data_layers/rasters/Landcover.tif,Landcover,Landcover,NA
8,rasters/Population_Density.tif,gs://live_data_layers/rasters/Population_Densi...,Population_Density,Population Density,"{""discrete"": ""FALSE"", ""sourceName"": ""2010 US C..."
9,rasters/Precipitation_mm.tif,gs://live_data_layers/rasters/Precipitation_mm...,Precipitation_mm,Precipitation mm,NA


In [11]:
import requests

def send_request(session, asset_name,props):
    # Earth Engine enabled Cloud Project.
    project_folder = 'ee-swhm'
    # A folder (or ImageCollection) name and the new asset name.
    asset_id = asset_name

    #url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId=production_layers/{}'
    url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId=production_layers/{}'
    params = {'overwrite': True}
    response = session.post(
      url = url.format(project_folder, asset_id),
      data = props, 
        params=params
    )
    
    print(json.loads(response.content))
    
    


In [12]:
import json
import pprint
def process_df(df): 
    for index, row in df.iterrows():
        json_string = row['json_data']
        print(row['pretty_name'])
        print('...')
        if json_string is not None and json_string != 'NA':
            layer = df['pretty_name'][index]
            asset_name = df['asset_name'][index]
            uri = df['uri'][index]

            # Request body as a dictionary.
            request = {
                'type': 'IMAGE',
                'gcs_location': {'uris': uri},
                'properties':json.loads(df['json_data'][index])
            }

            props = json.dumps(request)
            send_request(session, asset_name,props)


In [13]:
df.iloc[10:11]

,blob_name,uri,asset_name,pretty_name,json_data
10,rasters/Runoff_mm.tif,gs://live_data_layers/rasters/Runoff_mm.tif,Runoff_mm,Runoff mm,"{""discrete"": ""FALSE"", ""sourceName"": ""The Natur..."


## Send the request

Make the POST request to the Earth Engine [`projects.assets.create`](https://developers.google.com/earth-engine/reference/rest/v1alpha/projects.assets/create) endpoint.

In [14]:
process_df(df.iloc[10:11])

Runoff mm
...
{'type': 'IMAGE', 'name': 'projects/ee-swhm/assets/production_layers/Runoff_mm', 'id': 'projects/ee-swhm/assets/production_layers/Runoff_mm', 'properties': {'default_reduction': 'mean', 'description': 'Mean annual runoff calculated through continuous simulation for the period 1970-1999.', 'discrete': 'FALSE', 'pretty_name': 'Runoff mm', 'safe_name': 'runoff_mm', 'scale': 10, 'sourceName': 'The Nature Conservancy', 'units': 'mm/year'}, 'updateTime': '2023-04-04T19:09:51.703039Z', 'geometry': {'type': 'LineString', 'coordinates': [[-124.43759447579123, 49.313460696355776], [-124.90226598620926, 49.31346068119286], [-124.9022613893421, 46.59262893226524], [-124.43759447597434, 46.59289781526483], [-124.0725648775795, 46.59284500874787], [-123.70753534360895, 46.59289778195549], [-123.3093213508173, 46.592782683848796], [-122.87792276480306, 46.592845025823536], [-122.47970869273843, 46.59284505405395], [-122.11467919156566, 46.592897811277005], [-121.68328057234342, 46.59271

PATCH https://earthengine.googleapis.com/v1/{asset.name=projects/*/assets/**}

## Details on COG-backed assets

### Permissions
The ACLs of COG-backed Earth Engine assets and the underlying data are managed separately. If a COG-backed asset is shared in Earth Engine, it is the owner's responsibility to ensure that the data in GCS is shared with the same parties. If the data is not visible, Earth Engine will return an error of the form "Failed to load the GeoTIFF at `gs://my-bucket/my-object#123456`" (123456 is the generation of the object).

### Generations
When a COG-backed asset is created, Earth Engine reads the metadata of the TIFF in Cloud Storage and creates asset store entry. The URI associated with that entry must have a generation.  See the [object versioning docs](https://cloud.google.com/storage/docs/object-versioning) for details on generations. If a generation is specified (e.g., `gs://foo/bar#123`), Earth Engine will use it. If a generation is not specified, Earth Engine will use the latest generation of the object. 

That means that if the object in GCS is updated, Earth Engine will return a "Failed to load the GeoTIFF at `gs://my-bucket/my-object#123456`" error because the expected object no longer exists (unless the bucket enables  multiple object versions).  This policy is designed to keep metadata of the asset in sync with the metadata of the object.  

### Configuration
In terms of how a COG should be configured, the TIFF MUST be:

- Tiled, where the tile dimensions are either:
  - 16x16
  - 32x32
  - 64x64
  - 128x128
  - 256x256
  - 512x512
  - 1024x1024

- Arranged so that all IFDs are at the beginning.

For best performance:

- Use tile dimensions of 128x128 or 256x256.
- Include power of 2 overviews.

See [this page](https://cogeotiff.github.io/rio-cogeo/Advanced/#web-optimized-cog) for more details on an optimized configuration.